In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Load movie data
movies = pd.read_csv('/content/movies.csv')

# Load rating data
ratings = pd.read_csv('/content/ratings.csv')

# Merge movies and ratings on 'movieId'
data = pd.merge(ratings, movies, on='movieId')

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Create a user-item matrix
user_item_matrix = train_data.pivot_table(index='userId', columns='movieId', values='rating')

# Fill missing values with 0 (assuming no rating means a rating of 0)
user_item_matrix = user_item_matrix.fillna(0)

# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

# Function to get movie recommendations for a given user
def get_movie_recommendations(user_id, user_item_matrix, user_similarity, n=5):
    user_ratings = user_item_matrix.loc[user_id].values.reshape(1, -1)
    similarity_scores = cosine_similarity(user_ratings, user_item_matrix.values).flatten()

    # Get indices of movies sorted by similarity score
    movie_indices = similarity_scores.argsort()[::-1]

    # Exclude movies the user has already rated
    user_rated_movies = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index
    movie_indices = [idx for idx in movie_indices if idx not in user_rated_movies]

    # Return the top N recommended movie IDs
    recommendations = movie_indices[:n]
    return recommendations

# Example: Get movie recommendations for user with ID 1
user_id_to_recommend = 1
recommendations = get_movie_recommendations(user_id_to_recommend, user_item_matrix, user_similarity, n=5)

# Display the recommended movies
recommended_movies = movies[movies['movieId'].isin(recommendations)]['title'].unique()
print(f"Top 5 movie recommendations for user {user_id_to_recommend}: {recommended_movies}")


Top 5 movie recommendations for user 1: ['It Takes Two (1995)'
 'Like Water for Chocolate (Como agua para chocolate) (1992)'
 'Stuart Saves His Family (1995)']
